# Quickstart

Generate a sample point cloud for input:

In [ ]:
# Random seed for reproducibility
RANDOM_SEED = 42

# Point cloud shape
N_SAMPLES = 5
N_FEATURES = 2

In [ ]:
import numpy as np

np.random.seed(42)
points = np.random.rand(N_SAMPLES, N_FEATURES)

Run bounding box optimization on the point cloud:

In [ ]:
import bbo

bbout = bbo.run(points)

The output is a `bbo.output.BBOOutput` object, which is a dataclass containing the optimization results:

In [ ]:
type(bbout)

In [ ]:
bbout

The `box` attribute holds the coordinates of the vertices (corners) of the minimum-volume oriented bounding box (OBB) for `points` (2^n vertices for n-dimensional points):

In [ ]:
bbout.box

In [ ]:
assert bbout.box.shape == (2 ** N_FEATURES, N_FEATURES)

The volume of the OBB is stored in `volume`:

In [ ]:
bbout.volume

In [ ]:
assert bbout.volume.shape == ()

In [ ]:
recalculated_volume = np.prod(bbout.points.max(axis=-2) - bbout.points.min(axis=-2))
assert np.allclose(bbout.volume, recalculated_volume)

The `points` attribute holds the coordinates of the rotated points,
i.e., the input `points` rotated so that the OBB is the same as the axis-aligned bounding box (AABB):

In [ ]:
bbout.points

In [ ]:
assert bbout.points.shape == (N_SAMPLES, N_FEATURES)

The rotation matrix used to rotate the input `points` into output `bbout.points` is stored in the `rotation` attribute:

In [ ]:
bbout.rotation

In [ ]:
assert bbout.rotation.shape == (N_FEATURES, N_FEATURES)

In [ ]:
assert np.array_equiv(points @ bbout.rotation, bbout.points)

Optimization can also be performed in a vectorized/parallelized manner with any number of leading batch dimensions:

In [ ]:
BATCH_SHAPE = (13, 12, 11)
N_SAMPLES_BATCH = 10
N_FEATURES_BATCH = 3
point_batches = np.random.rand(*BATCH_SHAPE, N_SAMPLES_BATCH, N_FEATURES_BATCH)
point_batches.shape

In [ ]:
batch_bbout = bbo.run(point_batches)
batch_bbout

In [ ]:
assert batch_bbout.box.shape == (*BATCH_SHAPE, 2 ** N_FEATURES_BATCH, N_FEATURES_BATCH)

In [ ]:
assert batch_bbout.volume.shape == (*BATCH_SHAPE, )

In [ ]:
assert batch_bbout.points.shape == (*BATCH_SHAPE, N_SAMPLES_BATCH, N_FEATURES_BATCH)

In [ ]:
assert batch_bbout.rotation.shape == (*BATCH_SHAPE, N_FEATURES_BATCH, N_FEATURES_BATCH)

## Visualization

Define visualization functions:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial import ConvexHull

import bbo.util


def plot(points, bbout):

    is_2d = points.shape[1] == 2
    is_3d = points.shape[-1] == 3

    # Center points for clearer visualization
    points_before, points_after, obb_vertices = center_points(points, bbout)

    # Compute convex hull to plot
    hull_vertices_before = calculate_hull_vertices(points_before)
    hull_vertices_after = calculate_hull_vertices(points_after)

    # Compute AABB area & vertices
    aabb_area_before, aabb_vertices_before = calculate_aabb(points_before)
    aabb_area_after, aabb_vertices_after = calculate_aabb(points_after)

    # Plot figures
    if is_2d:
        # Append the first vertex to close the visualization loop
        obb_vertices = np.append(obb_vertices, [obb_vertices[0]], axis=0)
        fig, axs = plt.subplots(1, 2, figsize=(14, 6))
        plot_figure_2d(axs[0], points_before, hull_vertices_before, obb_vertices, aabb_vertices_before, aabb_area_before)
        plot_figure_2d(axs[1], points_after, hull_vertices_after, obb_vertices, aabb_vertices_after, aabb_area_after, is_after=True)
    else:
        fig = plt.figure(figsize=(14, 6))
        axs = [fig.add_subplot(1, 2, i+1, projection='3d') for i in range(2)]
        plot_figure(axs[0], points_before, obb_vertices)
        plot_figure(axs[1], points_after, obb_vertices, is_after=True)

    # Global limits (cover all points and OBB box)
    set_coordinate_axes(points_before, obb_vertices, aabb_vertices_before, aabb_vertices_after, points_after, axs=axs)

    # Add global legend above plots
    handles0, labels0 = axs[0].get_legend_handles_labels()
    handles1, labels1 = axs[1].get_legend_handles_labels()
    # Combine and deduplicate labels
    handles_dict = dict(zip(labels0, handles0))  # Start with left plot entries
    for h, l in zip(handles1, labels1):
        if l not in handles_dict:
            handles_dict[l] = h  # Add only new labels (e.g., AABB/OBB Frame)
    fig.legend(
        handles_dict.values(),
        handles_dict.keys(),
        loc='lower center',
        bbox_to_anchor=(0.5, -0.07) if is_2d else None,
        ncol=5,
        frameon=True
    )

    # Layout adjustments
    plt.tight_layout(rect=[0, 0.07, 1, 1] if is_3d else None)
    # Reserve space for legend above
    # plt.subplots_adjust(bottom=1.9)
    # Show the plot
    plt.show()
    return


def center_points(points, bbout):
    """Center points around the mean for clearer visualization."""
    points_before_mean = points.mean(axis=0)
    points_before = points - points_before_mean
    points_after = bbout.points - bbout.points.mean(axis=0)
    obb_vertices = bbout.box - points_before_mean
    return points_before, points_after, obb_vertices


def calculate_hull_vertices(points):
    """Calculate convex hull vertices for visualization purposes."""
    hull = ConvexHull(points)
    # Append the first vertex to close the visualization loop
    vertices = np.append(hull.vertices, [hull.vertices[0]])
    return vertices


def calculate_aabb(points):
    """Calculate AABB vertices and area for visualization purposes."""
    aabb_lower = np.min(points, axis=0)
    aabb_upper = np.max(points, axis=0)
    aabb_area = np.prod(aabb_upper - aabb_lower)
    aabb_vertices = bbo.util.box_vertices_from_bounds(aabb_lower, aabb_upper)
    # Append the first vertex to close the visualization loop
    aabb_box = np.append(aabb_vertices, [aabb_vertices[0]], axis=0)
    return aabb_area, aabb_box


def set_coordinate_axes(*point_sets, axs, margin: float = 0.05):
    # Match X and Y limits across subplots
    ndim = point_sets[0].shape[-1]
    for axis_idx, axis_name in zip(range(ndim), ['x', 'y', 'z']):
        all_values = np.concatenate([point_set[:, axis_idx] for point_set in point_sets])
        axis_margin = (all_values.max() - all_values.min()) * margin
        axis_limits = (all_values.min() - axis_margin, all_values.max() + axis_margin)
        for ax in axs:
            lim_setter = getattr(ax, f'set_{axis_name}lim')
            lim_setter(axis_limits)
            label_setter = getattr(ax, f'set_{axis_name}label')
            label_setter(axis_name)
    return


def plot_figure_2d(ax, points, hull_vertices, obb_vertices, aabb_vertices, aabb_area, is_after: bool = False):
    """Plot the points, convex hull, and AABB/OBB."""
    ax.scatter(points[:, 0], points[:, 1], color="black", label="Points", zorder=3)
    ax.plot(
        points[hull_vertices, 0],
        points[hull_vertices, 1],
        'r--',
        lw=1,
        label="Convex Hull",
        zorder=2
    )
    ax.plot(
        aabb_vertices[:, 0],
        aabb_vertices[:, 1],
        f'{"g" if is_after else "b"}-.',
        lw=2,
        label=f"{"AABB/OBB" if is_after else "AABB"} (area = {aabb_area:.2f})",
        zorder=1,
    )
    if not is_after:
        ax.plot(
            obb_vertices[:, 0],
            obb_vertices[:, 1],
            'g-',
            lw=2,
            label=f"OBB (area = {bbout.volume:.2f})",
            zorder=4
        )
    # Aspect ratio
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"{"Rotated" if is_after else "Original"} Points")
    return


def plot_box(ax, vertices, color, alpha, label):
    faces = [
        [0, 1, 2, 3], [4, 5, 6, 7],  # bottom, top
        [0, 1, 5, 4], [2, 3, 7, 6],  # front, back
        [0, 3, 7, 4], [1, 2, 6, 5]   # left, right
    ]
    poly3d = [[vertices[i] for i in face] for face in faces]
    collection = Poly3DCollection(poly3d, alpha=alpha, facecolor=color, edgecolor='k', linewidths=0.5, label=label)
    ax.add_collection3d(collection)


def plot_aabb(ax, points, is_after: bool = False):
    aabb_lower = points.min(axis=0)
    aabb_upper = points.max(axis=0)
    aabb_vertices = bbo.util.box_vertices_from_bounds(aabb_lower, aabb_upper)
    aabb_volume = np.prod(aabb_upper - aabb_lower)
    plot_box(ax, aabb_vertices, color='g' if is_after else 'c', alpha=0.2, label=f"{"AABB/OBB" if is_after else "AABB"} (volume = {aabb_volume:.2f})")
    return


def plot_figure(ax, points, obb_vertices, is_after: bool = False):
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=10, color="black", label="Points")
    hull = ConvexHull(points)
    for simplex in hull.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], points[simplex, 2], 'r--', label="Convex Hull", lw=0.5)
    plot_aabb(ax, points, is_after=is_after)
    if not is_after:
        plot_box(ax, obb_vertices, color='g', alpha=0.2, label=f"OBB (volume = {bbout.volume:.2f})")
    ax.set_title(f"{"Rotated" if is_after else "Original"} Points")
    return

### 2D Points

Generate a 2D point cloud as sample input:

In [ ]:
import arrayer

points = arrayer.pcloud.cylinder(
    n_points=30,
    radius=0.5,
    start=(-1, -1, -1),
    end=(1, 1, 1)
)[..., :2]
bbout = bbo.run(points)
plot(points, bbout)

### 3D Points

In [ ]:
%matplotlib widget
points = arrayer.pcloud.cylinder(radius=0.5, n_points=30, start=(-1, -1, -1), end=(1, 1, 1))
bbout = bbo.hull.run(points)
plot(points, bbout)